## **Pre-trained EfficientNetB0 Model**

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import glob

In [10]:
img_size = (224, 224)  # EfficientNetB0 default input size
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/work/AdrianAppeltAydas#1677/Solution Dataset/train/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='int'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/work/AdrianAppeltAydas#1677/Solution Dataset/val/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='int'
)

# Load the test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/work/AdrianAppeltAydas#1677/Solution Dataset/test/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='int'
)

Found 284900 files belonging to 2 classes.
Found 60157 files belonging to 2 classes.
Found 60157 files belonging to 2 classes.


In [11]:
def EN_preprocess_fn(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [12]:
AUTOTUNE = tf.data.AUTOTUNE # Prefetch the datasets for performance. This allows the data loading of the next batch to be done in parallel with model training from the current batch.

#import os

#train_cache_path = "AdrianAppeltAydas#1677/cache/train"
#os.makedirs(train_cache_path, exist_ok=True)
# Preprocess the datasets for CNN. Caching on disk.
EN_train_ds = train_ds.map(EN_preprocess_fn, num_parallel_calls=AUTOTUNE) \
                       .cache("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Cache/train") \
                       .shuffle(1000) \
                       .prefetch(AUTOTUNE)

#val_cache_path = "AdrianAppeltAydas#1677/cache/val"
#os.makedirs(val_cache_path, exist_ok=True)
EN_val_ds = val_ds.map(EN_preprocess_fn, num_parallel_calls=AUTOTUNE) \
                       .cache("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Cache/val") \
                       .prefetch(AUTOTUNE)

#test_cache_path = "AdrianAppeltAydas#1677/cache/test"
#os.makedirs(test_cache_path, exist_ok=True)
EN_test_ds = test_ds.map(EN_preprocess_fn, num_parallel_calls=AUTOTUNE) \
                       .cache("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Cache/test") \
                       .prefetch(AUTOTUNE)

In [5]:
# Define model-building function for binary classification
def build_model(freeze_base=True):
    inputs = layers.Input(shape=(224, 224, 3))

    # Load EfficientNetB0 with ImageNet weights and set freeze_base to True or False based on the parameter
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)
    base_model.trainable = not freeze_base

    # Add custom classification head
    x = layers.GlobalMaxPooling2D()(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    # Compile the model
    model = tf.keras.Model(inputs, outputs, name="EfficientNetB0_Binary")
    return model

In [4]:
import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2.0, alpha=0.90):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_weight = alpha_factor * K.pow((1 - p_t), gamma)
        return -K.mean(focal_weight * K.log(p_t))
    return loss

In [7]:
# Build and compile model (Phase 1 - base frozen)
EN_model = build_model(freeze_base=True)
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-4)
EN_model.compile(optimizer=optimizer, loss=focal_loss(gamma=2.0, alpha=0.90), metrics=["accuracy", tf.keras.metrics.Precision(name='precision', thresholds=0.5),
        tf.keras.metrics.Recall(name='recall', thresholds=0.5)])
EN_model.summary()


Model: "EfficientNetB0_Binary"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,055,972 (15.47 MB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
EN_model_checkpoint = ModelCheckpoint(
    filepath='/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_{epoch:02d}.keras',  # Save a new file for each epoch
    save_weights_only=False,                  # Save the full model
    save_freq='epoch',                        # Save after each epoch
    save_best_only=False,                     # Save every epoch, not just the best
    verbose=1
)

In [9]:
import json
import os

# Create a save folder path
history_dir = '/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Training History'
os.makedirs(history_dir, exist_ok=True)

In [ ]:
# Train top layers only
EN_model.fit(
    EN_train_ds,
    epochs=10,
    validation_data=EN_val_ds,
    verbose=1,
    callbacks=[EN_model_checkpoint]
)

Epoch 1/10


2025-05-15 19:35:53.415061: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 860 of 1000
2025-05-15 19:35:54.963360: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


7906/8904 ━━━━━━━━━━━━━━━━━━━━ 2:28 149ms/step - accuracy: 0.8685 - loss: 0.0200 - precision: 0.0145 - recall: 0.1134

2025-05-15 19:55:32.271464: W tensorflow/core/kernels/data/cache_dataset_ops.cc:333] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.8731 - loss: 0.0197 - precision: 0.0147 - recall: 0.1096
Epoch 1: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_01.keras


...

8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.9585 - loss: 0.0136 - precision: 0.0174 - recall: 0.0303
Epoch 3: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_03.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 1582s 176ms/step - accuracy: 0.9585 - loss: 0.0136 - precision: 0.0174 - recall: 0.0303 - val_accuracy: 0.9990 - val_loss: 0.0038 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.9711 - loss: 0.0125 - precision: 0.0144 - recall: 0.0124
Epoch 4: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_04.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 1581s 176ms/step - accuracy: 0.9711 - loss: 0.0125 - precision: 0.0144 - recall: 0.0124 - val_accuracy: 0.9990 - val_loss: 0.0040 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10


2025-05-15 21:23:00.084253: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 610 of 1000


   1/8904 ━━━━━━━━━━━━━━━━━━━━ 37:01:08 15s/step - accuracy: 1.0000 - loss: 0.0047 - precision: 0.0000e+00 - recall: 0.0000e+00

2025-05-15 21:23:04.814087: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.9748 - loss: 0.0121 - precision: 0.0101 - recall: 0.0061
Epoch 5: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_05.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 1559s 173ms/step - accuracy: 0.9748 - loss: 0.0121 - precision: 0.0101 - recall: 0.0061 - val_accuracy: 0.9990 - val_loss: 0.0043 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/10
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.9782 - loss: 0.0118 - precision: 0.0128 - recall: 0.0052
Epoch 6: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_06.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 1563s 174ms/step - accuracy: 0.9782 - loss: 0.0118 - precision: 0.0128 - recall: 0.0052 - val_accuracy: 0.9990 - val_loss: 0.0044 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 7/10
6292

In [18]:
import tensorflow as tf

# Load the model
from tensorflow.keras.models import load_model
EN_model = load_model("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/frozen_model_epoch_10.keras", compile=False) # Can't compile because there's a custom loss function (focal loss)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
EN_model_checkpoint = ModelCheckpoint(
    filepath='/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/unfrozen_model_epoch_{epoch:02d}.keras',  # Save a new file for each epoch
    save_weights_only=False,                  # Save the full model
    save_freq='epoch',                        # Save after each epoch
    save_best_only=False,                     # Save every epoch, not just the best
    verbose=1
)

In [20]:
# Recompile the model

import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2.0, alpha=0.90):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_weight = alpha_factor * K.pow((1 - p_t), gamma)
        return -K.mean(focal_weight * K.log(p_t))
    return loss

# Phase 2: Unfreeze base and fine-tune
for layer in EN_model.layers[-9:]:
    layer.trainable = True
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5)
EN_model.compile(optimizer=optimizer, loss=focal_loss(gamma=2.0, alpha=0.90), metrics=["accuracy", tf.keras.metrics.Precision(name='precision', thresholds=0.5),
        tf.keras.metrics.Recall(name='recall', thresholds=0.5)])

In [21]:
import json
import os

# Create a save folder path
history_dir = '/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss//Training History'
os.makedirs(history_dir, exist_ok=True)

In [ ]:
# Continue training the full model
EN_history = EN_model.fit(
    EN_train_ds,
    epochs=5,
    validation_data=EN_val_ds,
    verbose=1,
    callbacks=[early_stopping, EN_model_checkpoint]
)

EN_model.save("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/EN_final_model.keras")

# Save history as JSON
with open(os.path.join(history_dir, 'EN_history.json'), 'w') as f:
    json.dump(CNN_history.history, f)

Epoch 1/5
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.9725 - loss: 0.0121 - precision: 0.0157 - recall: 0.0117
Epoch 1: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/unfrozen_model_epoch_01.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 2265s 253ms/step - accuracy: 0.9725 - loss: 0.0121 - precision: 0.0157 - recall: 0.0117 - val_accuracy: 0.9990 - val_loss: 0.0055 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.9784 - loss: 0.0118 - precision: 0.0221 - recall: 0.0088
Epoch 2: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/unfrozen_model_epoch_02.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 2149s 240ms/step - accuracy: 0.9784 - loss: 0.0118 - precision: 0.0221 - recall: 0.0088 - val_accuracy: 0.9990 - val_loss: 0.0051 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Ep

2025-05-16 07:34:08.533108: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 745 of 1000


   1/8904 ━━━━━━━━━━━━━━━━━━━━ 36:24:00 15s/step - accuracy: 0.9688 - loss: 0.0149 - precision: 0.0000e+00 - recall: 0.0000e+00

2025-05-16 07:34:13.000764: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


8904/8904 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - accuracy: 0.9793 - loss: 0.0115 - precision: 0.0173 - recall: 0.0058
Epoch 4: saving model to /work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/unfrozen_model_epoch_04.keras
8904/8904 ━━━━━━━━━━━━━━━━━━━━ 2155s 240ms/step - accuracy: 0.9793 - loss: 0.0115 - precision: 0.0173 - recall: 0.0058 - val_accuracy: 0.9990 - val_loss: 0.0046 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/5


2025-05-16 08:10:03.633120: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 559 of 1000


   1/8904 ━━━━━━━━━━━━━━━━━━━━ 45:06:02 18s/step - accuracy: 0.9375 - loss: 0.0308 - precision: 0.0000e+00 - recall: 0.0000e+00

2025-05-16 08:10:11.662562: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


6650/8904 ━━━━━━━━━━━━━━━━━━━━ 7:30 200ms/step - accuracy: 0.9805 - loss: 0.0114 - precision: 0.0174 - recall: 0.0044

In [ ]:
import tensorflow as tf

# Load the model
from tensorflow.keras.models import load_model
EN_model = load_model("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Model Epoch Checkpoints/unfrozen_model_epoch_05.keras", compile=False) # Can't compile because there's a custom loss function (focal loss)

In [5]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-5)
EN_model.compile(optimizer=optimizer, loss=focal_loss(gamma=2.0, alpha=0.90), metrics=["accuracy", tf.keras.metrics.Precision(name='precision', thresholds=0.5),
        tf.keras.metrics.Recall(name='recall', thresholds=0.5)])

In [ ]:
import json

# Load training history from JSON
with open('/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/Training History/cnn_history.json', 'r') as f:
    loaded_history = json.load(f)

In [ ]:
plot_history(model.history)

In [ ]:
# Recompile the model

import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2.0, alpha=0.90):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_weight = alpha_factor * K.pow((1 - p_t), gamma)
        return -K.mean(focal_weight * K.log(p_t))
    return loss

optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss=focal_loss(gamma=2.0, alpha=0.90), metrics=["accuracy", tf.keras.metrics.Precision(name='precision', thresholds=0.5),
        tf.keras.metrics.Recall(name='recall', thresholds=0.5)])

In [13]:
loss, accuracy, precision, recall = EN_model.evaluate(EN_test_ds)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

1880/1880 ━━━━━━━━━━━━━━━━━━━━ 430s 226ms/step - accuracy: 0.9991 - loss: 0.0057 - precision: 0.0000e+00 - recall: 0.0000e+00
Loss: 0.0057
Accuracy: 0.9990
Precision: 0.0000
Recall: 0.0000


2025-05-16 09:07:51.046906: W tensorflow/core/kernels/data/cache_dataset_ops.cc:333] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [15]:
# Fine-tune the classification threshold on the validation dataset

from sklearn.metrics import recall_score
import numpy as np

# Predictions and true labels
predictions = EN_model.predict(EN_val_ds).flatten()
true_classes = np.concatenate([y.numpy() for _, y in EN_val_ds], axis=0)

# Thresholds to test
thresholds = np.arange(0.0, 1.001, 0.001)

recalls_class_0 = []
recalls_class_1 = []

# Evaluate each threshold
for thresh in thresholds:
    preds = (predictions >= thresh).astype(int)
    recall_per_class = recall_score(true_classes, preds, average=None, zero_division=0)
    recalls_class_0.append(recall_per_class[0])
    recalls_class_1.append(recall_per_class[1])


# Convert to arrays
recalls_class_0 = np.array(recalls_class_0)
recalls_class_1 = np.array(recalls_class_1)

# Find threshold where the difference in recall is minimized
recall_diff = np.abs(recalls_class_0 - recalls_class_1)
crossing_idx = np.argmin(recall_diff)
crossing_threshold = thresholds[crossing_idx]
recall0 = recalls_class_0[crossing_idx]
recall1 = recalls_class_1[crossing_idx]

# Print results
print(f"Crossing threshold: {crossing_threshold:.3f}")
print(f"Recall for class 0 (benign): {recall0:.4f}")
print(f"Recall for class 1 (malignant): {recall1:.4f}")


1880/1880 ━━━━━━━━━━━━━━━━━━━━ 425s 226ms/step


2025-05-16 09:15:44.236222: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Crossing threshold: 0.351
Recall for class 0 (benign): 0.5484
Recall for class 1 (malignant): 0.5000


In [17]:
# Make predictions on the test dataset
import numpy as np
class_names = test_ds.class_names
predictions = EN_model.predict(EN_test_ds)
predicted_classes = (predictions > crossing_threshold).astype("int32")
true_classes = np.concatenate([y.numpy() for _, y in EN_test_ds], axis=0)
from sklearn.metrics import classification_report, confusion_matrix

report = classification_report(true_classes, predicted_classes, target_names=class_names)

# Print classification report
print(report)

# Save to file
with open("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/EN_classification_report.txt", "w") as f:
    f.write(report)

1880/1880 ━━━━━━━━━━━━━━━━━━━━ 427s 226ms/step
              precision    recall  f1-score   support

      benign       1.00      0.55      0.71     60099
   malignant       0.00      0.48      0.00        58

    accuracy                           0.55     60157
   macro avg       0.50      0.52      0.35     60157
weighted avg       1.00      0.55      0.71     60157



2025-05-16 09:32:04.760088: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)
#np.save("/work/AdrianAppeltAydas#1677/EfficientNet/5th Run: EfficientNet with focal loss/EN_confusion_matrix_3.npy", conf_matrix)

[[32934 27165]
 [   30    28]]
